In [48]:
import warnings

import geopandas

warnings.filterwarnings('ignore')
geopandas.options.io_engine = "pyogrio"
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
from sqlalchemy import create_engine
from config import PostgresReadOnlyConfig

conn_str = PostgresReadOnlyConfig().__str__()
engine = create_engine(conn_str)


def water_quality_interval_query(parameters: list[str], locations: list[str], parameter: str, unit: str):
    parameters_csv = ','.join(f"'{p}'" for p in parameters)
    locations_csv = ','.join(f"'{l}'" for l in locations)

    query_template = f"""
        with 
        sample_date as (
             select wq.sample_date_time::date as sample_date from public.water_quality wq 
             where parameter IN ({parameters_csv})
                and wq.sample_location in ({locations_csv})
        ),
        intervals as (
            select 
                (select min(s.sample_date) from sample_date s) + ( n    || ' day')::interval start_date,
                (select min(s.sample_date) from sample_date s) + ((n+120) || ' day')::interval end_date
              from generate_series(
              0, 
              ((select max(s.sample_date)::date from sample_date s) - (select min(s.sample_date)::date from sample_date s)),
              120) n
          )
          SELECT 
            max(sample_location) as sample_location,
            max(wq.geometry) as geometry,
            max("parameter") as "parameter",
            avg(wq.value) as avg_value,
            max(wq.unit) as unit,
           i.start_date, 
           i.end_date
           FROM public.water_quality wq 
           right join intervals i on sample_date_time::date >= i.start_date and sample_date_time::date < i.end_date
           where parameter IN ({parameters_csv})
            and wq.sample_location in ({locations_csv})
            group by i.start_date, i.end_date
            order by i.start_date desc;
    """

    df = geopandas.read_postgis(
        sql=query_template,
        con=engine, geom_col='geometry', crs="EPSG:26914")
    df["unit"] = unit
    df["parameter"] = parameter
    return df


In [50]:
import pandas
import itertools

twenty_fourth_st = ["Shoal Creek @ 24th Street"]
twelfth_st = ["USGS-08156800"]
upstream_fist_st = ["Shoal Creek Upstream of 1st St"]

coa_parameters = {
    "tss": ["TOTAL SUSPENDED SOLIDS"],
    # "tds": ["TOTAL DISSOLVED SOLIDS"],
    "ph": ["PH"],
    "temperature": ["WATER TEMPERATURE"],
    "conductivity": ["CONDUCTIVITY"],
    "turbidity": ["TURBIDITY"], 
    "n0": ["NITRATE AS N"],
    "p04": ["PHOSPHORUS AS P", "ORTHOPHOSPHORUS AS P"]
}
usgs_parameters = {
    "n0": ["Inorganic nitrogen (nitrate and nitrite)", "Kjeldahl nitrogen", "Organic Nitrogen", "Nitrite", "Nitrate", "Nitrogen, mixed forms (NH3), (NH4), organic, (NO2) and (NO3)"],
    "p04": ["Phosphorus", "Orthophosphate"],
    "ph": ["pH"],
    # "tds": ["Total dissolved solids"],
    "turbidity": ["Turbidity"],
    "conductivity": ["Specific conductance"],
    "tss": ["Total suspended solids"],
    "temperature": ["Temperature, water"]
}

units = {
    "n0": "mg/l",
    "p04": "mg/l",
    "ph": "std units",
    # "tds": "ppm",
    "turbidity": "NTU",
    "conductivity": "uS/cm",
    "tss": "MG/L",
    "temperature": "deg C"
}

twenty_fourth_st_intervals = geopandas.GeoDataFrame()
for parameter in usgs_parameters:
    df = water_quality_interval_query(parameters=coa_parameters[parameter], locations=twelfth_st, unit=units[parameter], parameter=parameter)
    twenty_fourth_st_intervals = pandas.concat([twenty_fourth_st_intervals, df])

twelfth_st_intervals = geopandas.GeoDataFrame()
for parameter in usgs_parameters:
    df = water_quality_interval_query(parameters=usgs_parameters[parameter], locations=twelfth_st, unit=units[parameter], parameter=parameter)
    twelfth_st_intervals = pandas.concat([twelfth_st_intervals, df])


upstream_fist_st_intervals = geopandas.GeoDataFrame()
for parameter in usgs_parameters:
    df =  water_quality_interval_query(parameters=coa_parameters[parameter], locations=twelfth_st, unit=units[parameter], parameter=parameter)
    upstream_fist_st_intervals = pandas.concat([upstream_fist_st_intervals, df])


twelfth_st_intervals


,sample_location,geometry,parameter,avg_value,unit,start_date,end_date
0,USGS-08156800,POINT (-97.750 30.277),n0,1.647333,mg/l,2018-02-05,2018-06-05
1,USGS-08156800,POINT (-97.750 30.277),n0,0.917500,mg/l,2017-10-08,2018-02-05
2,USGS-08156800,POINT (-97.750 30.277),n0,0.988375,mg/l,2017-06-10,2017-10-08
3,USGS-08156800,POINT (-97.750 30.277),n0,0.806312,mg/l,2016-02-16,2016-06-15
4,USGS-08156800,POINT (-97.750 30.277),n0,0.857533,mg/l,2015-10-19,2016-02-16
...,...,...,...,...,...,...,...
33,USGS-08156800,POINT (-97.750 30.277),temperature,26.000000,deg C,1983-08-08,1983-12-06
34,USGS-08156800,POINT (-97.750 30.277),temperature,10.250000,deg C,1982-12-11,1983-04-10
35,USGS-08156800,POINT (-97.750 30.277),temperature,11.000000,deg C,1982-04-15,1982-08-13
36,USGS-08156800,POINT (-97.750 30.277),temperature,21.000000,deg C,1981-04-20,1981-08-18
